In [1]:
# import the modules
import pandas as pd
import numpy as np

## Read the data

In [2]:
# Reading all the datasets
application_test = pd.read_csv('application_test.csv')
application_train = pd.read_csv('application_train.csv') 
bureau = pd.read_csv('bureau.csv')
bureau_balance = pd.read_csv('bureau_balance.csv')
credit_card_balance = pd.read_csv('credit_card_balance.csv')
HomeCredit_columns_description = pd.read_csv('HomeCredit_columns_description.csv',encoding = 'ISO-8859-1', index_col = 0)
installments_payments = pd.read_csv('installments_payments.csv',encoding = 'ISO-8859-1')
POS_CASH_balance = pd.read_csv('POS_CASH_balance.csv',encoding = 'ISO-8859-1')
previous_application = pd.read_csv('previous_application.csv',encoding = 'ISO-8859-1')
sample_submission = pd.read_csv('sample_submission.csv',encoding = 'ISO-8859-1')

## 1. average requested loan size (current and previous) for all applicants

Merge: application_train, bureaus, previous 

In [4]:
# Perform the merging
results1 = application_train[['SK_ID_CURR', 'AMT_CREDIT']].merge(bureau, on = 'SK_ID_CURR', suffixes = ('_train', '_bureau'))\
        .merge(previous_application, on = 'SK_ID_CURR', suffixes = ('', '_previous'), how = 'outer')


In [5]:
# Displaying the results of the exercise
results1.groupby('SK_ID_BUREAU').agg({'AMT_CREDIT_previous': 'mean', 'AMT_CREDIT': 'mean'}).mean()

AMT_CREDIT_previous    181907.064386
AMT_CREDIT             623597.933385
dtype: float64

Conclusion: Previous average loan size: 181907. Current average loan size: 623597

## 2. how many applicants from the application_train table were previously rejected
Merge: application_train, bureaus, previous  

In [25]:
# This will merge the application train with the bureau and previous application
# Then it queries based on NAME_CONTRACT_STATUS to get Refused applications
# It will groupby SK_ID_BUREAU and ngroups attribute to get the amount of applicants rejected
result = application_train[['SK_ID_CURR', 'AMT_CREDIT']].merge(bureau, on = 'SK_ID_CURR', suffixes = ('_train', '_bureau'))\
.merge(previous_application, on = 'SK_ID_CURR', suffixes = ('', '_previous')) \
.query('NAME_CONTRACT_STATUS == "Refused"')\
.groupby('SK_ID_BUREAU').ngroups

print(f"Number of previously reject applicants: {result}")

Number of previously reject applicants: 579424


##  3. average CURRENT credit card balance for the applicants
Merge: application_train, bureaus, bureas_balance

In [15]:
# Merging the bureau with application train and credit card balance
# Set the query to MONTHS_BALANCE == -1 to get the most current data
# Group by SK_ID_BUREAU and get the mean AMT balance to get the average CC balance
# Sorting by AMT_BALANCE from most to least
bureau[['SK_ID_CURR', 'SK_ID_BUREAU']].merge(application_train[['SK_ID_CURR', 'AMT_CREDIT']]
                                             , on = 'SK_ID_CURR', suffixes = ('_bureau', '_train'))\
.merge(credit_card_balance[['SK_ID_CURR', 'MONTHS_BALANCE', 'AMT_BALANCE']], on = 'SK_ID_CURR')\
.query('MONTHS_BALANCE == -1')\
.groupby('SK_ID_BUREAU').agg({'AMT_BALANCE': 'mean'})\
.sort_values('AMT_BALANCE', ascending = False)

,AMT_BALANCE
SK_ID_BUREAU,
5358176,1210510.755
5358178,1210510.755
5358173,1210510.755
5358174,1210510.755
5358175,1210510.755
...,...
5687385,0.000
5687405,0.000
5687406,0.000


## 4. ratio of approved/refused loans per gender? 
Merge: application_train, previous


In [16]:
# Merging the tables train and previous_application
# Group by CODE_GENDER and get the percentage of NAME_CONTRACT_STATUS
application_train[['SK_ID_CURR', 'CODE_GENDER']].merge(previous_application, on = 'SK_ID_CURR')\
.groupby('CODE_GENDER')['NAME_CONTRACT_STATUS'].value_counts(normalize = True)

CODE_GENDER  NAME_CONTRACT_STATUS
F            Approved                0.626742
             Canceled                0.187446
             Refused                 0.170972
             Unused offer            0.014840
M            Approved                0.626926
             Refused                 0.178979
             Canceled                0.175345
             Unused offer            0.018750
XNA          Refused                 0.454545
             Approved                0.418182
             Canceled                0.127273
Name: NAME_CONTRACT_STATUS, dtype: float64

### Conclusion
Loans approved/refused ratio per gender:
- Females: 62,7% approved vs 17,1% refused
- Males: 62,7% approved vs 17,9% refused

## 5. Do people with children have a higher or lower loan approval ratio than those without
Merge: application_train, previous

In [10]:
# Assign a new feature: HAS_CHILDREN. Yes if the applicant has children, no if not
# Merge with the previous application data
# Groupby HAS_CHILDREN and access the name of the contract status
# Get the percentage for each group and filter to only get the approved 
application_train.assign(HAS_CHILDREN = np.where(application_train['CNT_CHILDREN'] > 0, "Yes", "No")) \
.merge(previous_application, on = 'SK_ID_CURR')\
.groupby('HAS_CHILDREN')['NAME_CONTRACT_STATUS'] \
.value_counts(normalize = True).filter(like ='Approved', axis = 0)

HAS_CHILDREN  NAME_CONTRACT_STATUS
No            Approved                0.618859
Yes           Approved                0.646297
Name: NAME_CONTRACT_STATUS, dtype: float64

Conclusion: people with children have a higher loan approval ratio of about 3 percent

## 6. Bin client ages in young, middle age and senior citizens -- and create a pivot table showing percentages of loans Approved, Cancelled and Refused per age group.
young: 20-35 years (7300-12775 day)
middle: 35-60 years (12775-21000 days)
senior: 60+ years old (21000+ days)

In [13]:
# Setting the conditions
# Because the DAYS_BIRTH are negative, the conditions have to be treated that way
conditions = [
    (application_train['DAYS_BIRTH'] > -12775),
    (application_train['DAYS_BIRTH'] <= -12775) & (application_train['DAYS_BIRTH'] >= -21000),
    (application_train['DAYS_BIRTH'] < -21000)
]
# Setting the values for age group
values = ["young", "middle", "senior"]

# Setting the data merge
data = application_train.assign(AGE_GROUP = np.select(conditions, values))\
.merge(previous_application, on = 'SK_ID_CURR')

# Setting the pivot table
# Use the len function, after that devide the numbers by the total for the age group and multiply by 100 using mul
# E.g. (portion middle / total middle) * 100
# Select every row and column except the last column
pivot = data.pivot_table(columns = 'AGE_GROUP', index = 'NAME_CONTRACT_STATUS', values = 'SK_ID_CURR', 
                         aggfunc = len, margins = True)
pivot.div(pivot.iloc[-1]).mul(100).iloc[:, :-1]

AGE_GROUP,middle,senior,young
NAME_CONTRACT_STATUS,,,
Approved,62.887861,62.081850,62.655008
Canceled,18.512946,20.991792,16.051586
Refused,17.053415,16.218316,18.874083
Unused offer,1.545778,0.708042,2.419323
All,100.000000,100.000000,100.000000


## 7. Find clients that have the most recorded loans in the Credit bureau (above 95th percentile). What percentage of those have had at least one loan with STATUS == 5 (DPD 120+) in their Credit Bureau history